In [1]:
# !pip install transformers datasets evaluate peft

In [2]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig
from peft import LoraConfig, get_peft_model

# LoRA 설정
lora_config = LoraConfig(
    r=8,  # Low-rank의 차원
    lora_alpha=16,  # LoRA alpha
    lora_dropout=0.1,  # Dropout 비율
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj","v_proj","o_proj"],
)

# 멀티태스크 모델 정의
class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super(MultiTaskModel, self).__init__()
        config = AutoConfig.from_pretrained(model_name)
        config.output_hidden_states = True

        # 사전 학습된 모델 로드
        self.lm_model = AutoModelForCausalLM.from_pretrained(model_name, config=config)
        self.lm_model.config.pad_token_id = 128001
        self.lm_model = get_peft_model(self.lm_model, lora_config)
        hidden_size = self.lm_model.config.hidden_size

        # 분류 헤드 추가
        # self.classifier = nn.Linear(hidden_size, num_classes)
        intermediate_size = hidden_size // 2  # 중간 크기 설정
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, intermediate_size),  # 첫 번째 선형층
            nn.ReLU(),  # 활성화 함수
            nn.Linear(intermediate_size, num_classes)  # 두 번째 선형층
        )

    def forward(self, lm_input_ids, lm_attention_mask, class_input_ids=None, class_attention_mask=None, labels=None, class_labels=None):

        if class_input_ids is None:
            class_input_ids = lm_input_ids
        if class_attention_mask is None:
            class_attention_mask = lm_attention_mask

        # 언어 모델링 출력
        outputs = self.lm_model(input_ids=lm_input_ids, attention_mask=lm_attention_mask, labels=labels)
        lm_loss = outputs.loss  # LM 손실
        logits = outputs.logits  # LM 결과

        # 분류 헤드 출력 (CLS 토큰 사용)
        class_outputs = self.lm_model(input_ids=class_input_ids, attention_mask=class_attention_mask)
        hidden_states = class_outputs.hidden_states  # Hidden states 확인
        pooled_output = torch.mean(hidden_states[-1], dim=1)  # Mean Pooling
        class_logits = self.classifier(pooled_output)

        # 분류 손실 계산
        if class_labels is not None:
            class_loss = nn.CrossEntropyLoss()(class_logits, class_labels)
        else:
            class_loss = None

        return lm_loss, class_logits, class_loss


In [3]:
from tqdm import tqdm
import torch.nn.functional as F

# 데이터 정리 및 패딩
def collate_fn(batch):
    lm_input_ids = torch.nn.utils.rnn.pad_sequence([b["lm_input_ids"] for b in batch], batch_first=True, padding_value=0)
    lm_attention_mask = torch.nn.utils.rnn.pad_sequence([b["lm_attention_mask"] for b in batch], batch_first=True, padding_value=0)
    class_input_ids = torch.nn.utils.rnn.pad_sequence([b["class_input_ids"] for b in batch], batch_first=True, padding_value=0)
    class_attention_mask = torch.nn.utils.rnn.pad_sequence([b["class_attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence([b["labels"] for b in batch], batch_first=True, padding_value=-100)
    class_labels = torch.tensor([b["class_labels"] for b in batch])
    return {"lm_input_ids": lm_input_ids, "lm_attention_mask": lm_attention_mask,"class_input_ids":class_input_ids,"class_attention_mask":class_attention_mask, "labels": labels, "class_labels": class_labels}


# 학습 루프
def train(model, train_dataloader, eval_dataloader, optimizer, alpha=0.5, epochs=3):
    model.train()

    for epoch in range(epochs):
        total_loss = 0.0
        train_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Training]")  # tqdm 추가

        for batch in train_bar:
            lm_input_ids = batch["lm_input_ids"].to(device)
            lm_attention_mask = batch["lm_attention_mask"].to(device)
            class_input_ids = batch["class_input_ids"].to(device)
            class_attention_mask = batch["class_attention_mask"].to(device)
            labels = batch["labels"].to(device)
            class_labels = batch["class_labels"].to(device)

            # 모델 출력
            lm_loss, class_logits, class_loss = model(lm_input_ids, lm_attention_mask, class_input_ids, class_attention_mask, labels, class_labels)

            # 총 손실 계산
            if class_loss is not None:
                loss = lm_loss + alpha * class_loss
            else:
                loss = lm_loss

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 손실값 업데이트
            total_loss += loss.item()
            train_bar.set_postfix({"Loss": total_loss / (train_bar.n + 1)})
            print(f"LM Loss: {lm_loss.item():.4f}, Class Loss: {class_loss.item():.4f}")


        print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {total_loss / len(train_dataloader)}")

        # 검증 데이터 평가
        evaluate(model, eval_dataloader)


# 검증 루프
# 검증 루프
def evaluate(model, eval_dataloader):
    model.eval()
    total_lm_loss = 0.0
    total_correct = 0
    total_samples = 0

    # 데이터로더 비어 있는지 확인
    if len(eval_dataloader) == 0:
        print("Warning: Eval dataloader is empty!")
        return

    eval_bar = tqdm(eval_dataloader, desc="[Evaluating]")  # tqdm 추가

    with torch.no_grad():
        for batch in eval_bar:
            lm_input_ids = batch["lm_input_ids"].to(device)
            lm_attention_mask = batch["lm_attention_mask"].to(device)
            class_input_ids = batch["class_input_ids"].to(device)
            class_attention_mask = batch["class_attention_mask"].to(device)
            labels = batch["labels"].to(device)
            class_labels = batch["class_labels"].to(device)

            # 모델 출력
            lm_loss, class_logits, _ = model(lm_input_ids, lm_attention_mask, class_input_ids, class_attention_mask, labels)

            # NaN 체크
            if torch.isnan(lm_loss):
                print("Warning: NaN detected in LM Loss!")
                continue

            # 언어 모델링 손실 계산
            total_lm_loss += lm_loss.item()

            # 분류 정확도 계산
            # class_logits의 마지막 차원을 기준으로 argmax를 취함
            predictions = torch.argmax(class_logits, dim=-1)

            # 정확도 계산
            correct = (predictions == class_labels).sum().item()
            total_correct += correct
            total_samples += class_labels.size(0)

    # 평균 LM 손실 및 정확도 계산
    avg_lm_loss = total_lm_loss / len(eval_dataloader)
    accuracy = total_correct / total_samples

    print(f"Validation - LM Loss: {avg_lm_loss:.4f}, Classification Accuracy: {accuracy:.4f}")

    model.train()  # 평가 후 다시 학습 모드로 전환

In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# 1. CSV 파일 읽기
train_df = pd.read_csv(r"./05_sentence.csv")
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

eval_df = pd.read_csv(r"./05_sentence_eval.csv")
eval_df = eval_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df.head()

,skey,ti,ab,clr,cla,ipcs,label_idx
0,4920013002569,photocatalytic roofing granules photocatalytic...,photocatalytic roofing granules include a bind...,1 a process for preparing a photocatalytic roo...,1 a process for preparing a photocatalytic roo...,C09C C01G C09C B82Y E04D C01G,4
1,4920035002591,nonequilibrium polaronic quantum phasecondensa...,electrical devices operating in a range of 273...,1 an electrical device comprising a an insulat...,1 an electrical device comprising a an insulat...,H01L H01S C01G C01G H01L H01L H01L H01L H01P H...,4
2,4920031002499,methods of producing composite zeolite catalys...,a method of forming composite zeolite catalyst...,1 a method of forming composite zeolite cataly...,1 a method of forming composite zeolite cataly...,C01B B01J C01B B01J B01J B01J B01J B01J C01B B...,0
3,4919017002322,method for the generation of power,disclosed herein is a method comprising heatin...,1 a method comprising heating a strontiumconta...,1 a method comprising heating a strontiumconta...,F03G C01B C01F C01F F28F F03G C01B,3
4,4920030001982,method of preparing mesoporous carbon material...,the present invention relates to a method of p...,1 a method of preparing a mesoporous carbon ma...,1 a method of preparing a mesoporous carbon ma...,C01B B33Y B33Y B29C,0


In [5]:
# 데이터 로더
def create_train_data(tokenizer, ti, ab, label):
    # Chat 템플릿 생성
    chat = [
        {"role": "system", "content":"You are an AI that generates classification labels for patents. Read the text and classify them correctly. Let's Think Step by Step."},
        {"role": "user", "content": f"Generate a label for the given text.\n'{ti} {ab}'"},
        {"role": "assistant", "content": f"Patent classification label is {label_dict[label]}"},  # 정답 텍스트
    ]

    # Chat 형식에 맞춰 텍스트 생성
    input_txt = tokenizer.apply_chat_template(chat, tokenize=False)

    # 토큰화
    input_ids = tokenizer.encode(input_txt, return_tensors="pt", max_length=1024, truncation=True).squeeze(0)
    attention_mask = torch.ones_like(input_ids)

    class_input_txt = tokenizer.apply_chat_template([chat[0],chat[1]], tokenize=False)
    class_input_ids = tokenizer.encode(class_input_txt, return_tensors="pt", max_length=1024, truncation=True).squeeze(0)
    class_attention_mask = torch.ones_like(class_input_ids)


    # 토큰화
    input_ids = tokenizer.encode(input_txt, return_tensors="pt", max_length=1024, truncation=True).squeeze(0)
    attention_mask = torch.ones_like(input_ids)

    # 정답 토큰 인덱스 식별
    # 'assistant'의 정답 범위를 마스킹하기 위해 위치 식별
    assistant_text = f"Patent classification label is {label_dict[label]}"
    assistant_start = input_txt.index(assistant_text)  # assistant 정답이 시작되는 위치
    assistant_ids = tokenizer.encode(assistant_text, add_special_tokens=False)

    # 마스킹 처리
    labels = input_ids.clone()

    # 모든 토큰을 -100으로 설정
    labels[:] = -100

    # assistant 출력 범위만 정답으로 설정
    assistant_start_idx = (input_ids == assistant_ids[0]).nonzero(as_tuple=True)[0].item()  # 시작 토큰 인덱스
    labels[assistant_start_idx:assistant_start_idx + len(assistant_ids)] = input_ids[assistant_start_idx:assistant_start_idx + len(assistant_ids)]

    # 클래스 라벨은 텍스트의 분류 결과를 나타내므로, class_label로 별도로 처리
    class_label = label

    return {"lm_input_ids": input_ids, "lm_attention_mask": attention_mask,"class_input_ids":class_input_ids,"class_attention_mask":class_attention_mask, "labels": labels, "class_labels": class_label}

In [6]:
label_dict = {0:"Class_01B",1:"Class_01C",2:"Class_01D",3:"Class_01E",4:"Class_01F",5:"Class_01G"}

In [7]:
# 모델 초기화 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 분류 클래스 수
num_classes = 5

model_name = './일반 모델/Meta-Llama-3.1-8B-Instruct'
model = MultiTaskModel(model_name, num_classes).to(device)

for parameter in model.classifier.parameters():
    parameter.requires_grad = True

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# 토크나이저 및 데이터 준비
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = []
for i in train_df.iterrows():
    train_dataset.append(create_train_data(tokenizer, i[1]['ti'],i[1]['ab'],i[1]['label_idx']))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)

eval_dataset = []
for i in eval_df.iterrows():
    eval_dataset.append(create_train_data(tokenizer,i[1]['ti'], i[1]['ab'],i[1]['label_idx']))
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=2, collate_fn=collate_fn)

# 옵티마이저 설정
optimizer = torch.optim.AdamW([
    {"params": model.lm_model.parameters(), "lr": 5e-6},  # 언어 모델은 로스는 0.2 정도
    {"params": model.classifier.parameters(), "lr": 5e-4}  # 분류 헤드 로스는 높게 0.1이하가 되도록
])
# 학습 실행
train(model, train_dataloader, eval_dataloader, optimizer, alpha=1, epochs=3)

Epoch 1/3 [Training]:   0%|          | 1/400 [00:01<08:50,  1.33s/it, Loss=7.22]

LM Loss: 5.9255, Class Loss: 1.2986


Epoch 1/3 [Training]:   0%|          | 2/400 [00:02<06:21,  1.04it/s, Loss=13.4]

LM Loss: 5.7842, Class Loss: 13.7080


Epoch 1/3 [Training]:   1%|          | 3/400 [00:02<05:34,  1.19it/s, Loss=13.2]

...

LM Loss: 0.0802, Class Loss: 0.6072


Epoch 3/3 [Training]: 100%|██████████| 400/400 [04:36<00:00,  1.45it/s, Loss=0.504]


LM Loss: 0.0633, Class Loss: 0.0896
Epoch 3/3, Average Loss: 0.5044455563271185


[Evaluating]: 100%|██████████| 100/100 [00:34<00:00,  2.91it/s]

Validation - LM Loss: 0.0872, Classification Accuracy: 0.8600


In [9]:
def predict_class(model, tokenizer, text, max_length=1024):
    model.eval()
    
    chat = [
    {"role": "system", "content":"You are an AI that generates classification labels for patents. Read the text and classify them correctly. Let's Think Step by Step."},
    {"role": "user", "content": f"Generate a label for the given text.\n'{text}'"}, # 클래스분류 훈련 사용
    ]

    # Chat 형식에 맞춰 텍스트 생성
    input_txt = tokenizer.apply_chat_template(chat, tokenize=False)

    # 텍스트 토큰화
    inputs = tokenizer(input_txt, return_tensors="pt", max_length=max_length, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 모델 출력
    with torch.no_grad():
        _, class_logits, _ = model(input_ids, attention_mask, input_ids, attention_mask)  # class_logits는 분류 예측 값

    # 예측된 클래스
    predictions = torch.argmax(class_logits, dim=-1)

    return predictions.item()  # 예측된 클래스 반환


In [16]:
def generate_text(model, tokenizer, prompt, label, max_length=1024):
    model.eval()

    chat = [
    {"role": "system", "content":"You are an AI that generates classification labels for patents. Read the text and classify them correctly. Let's Think Step by Step."},
    {"role": "user", "content": f"Generate a label for the given text.\n'{prompt}'"}, # 클래스분류 훈련 사용
    {"role": "assistant", "content": f"Patent classification label is {label}"}, # LM 훈련 사용
    {"role": "user", "content": f"Why?"}, # 두가지 훈련으로 생겨나는 추가 기능?
    ]


    # Chat 형식에 맞춰 토큰화
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
    input_ids = inputs.to(device)

    # 텍스트 생성
    with torch.no_grad():
        generated_ids = model.lm_model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            # repetition_penalty = 1,
            top_p=0.95,
            temperature=0.6
        )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return generated_text


In [17]:
i = 33

predict_list = []
true_list = []
generated_text_list = []

for  i in range(len(eval_df)):
    text = f"{eval_df.iloc[i]['ti']} {eval_df.iloc[i]['ab']}'"

    predicted_class = predict_class(model, tokenizer, text)

    print(f"")
    print(f"Input Text: {text}")

    print(f"")
    print(f"Predicted Class: {predicted_class}")
    predict_list.append(predicted_class)

    print(f"")
    print(f"True Class: {eval_df.iloc[i]['label_idx']}")
    true_list.append(eval_df.iloc[i]['label_idx'])

    generated_text = generate_text(model, tokenizer, text, label_dict[predicted_class])

    print(f"")
    print(f"Generated Text: {generated_text.split('assistant')[-1]}")
    generated_text_list.append(generated_text.split('assistant')[-1])
    print("_____"*30)


Input Text: production of crystalline sodium bicarbonate a process for producing crystalline sodium bicarbonate comprising providing an aqueous sodiumbicarbonate containing liquor originating from a reactive crystallization step in which first sodium bicarbonate crystals are produced and recovered feeding at least a portion of said aqueous sodiumbicarbonate containing liquor to a cooling crystallization unit to form second sodium bicarbonate crystals and produce a crystals slurry comprising the second sodium bicarbonate crystals and withdrawing a portion of the crystals slurry from the cooling crystallization unit for the withdrawn second sodium bicarbonate crystals to be further processed a portion of the second sodium bicarbonate crystals withdrawn from the cooling crystallization unit may be fed to a sodium bicarbonate reactive crystallization unit to a caustic unit or may be separated and dried the reactive crystallization separation andor drying units where the second sodium bica

In [18]:
import pandas as pd

df = pd.DataFrame({
    'predict': predict_list,
    'true': true_list,
    'generated_text': generated_text_list
})

# 정답율 계산
accuracy = (df['predict'] == df['true']).mean()

print("DataFrame:")
print(df)
print(f"\n정답율: {accuracy:.2%}")

DataFrame:
     predict  true                                     generated_text
0          2     2  \n\nThe text describes a process for producing...
1          0     0  \n\nThe label Class_01B is a classification fo...
2          0     0  \n\nThe label Class_01B is a classification fo...
3          3     3  \n\nThe label Class_01E represents the classif...
4          2     3  \n\nThe patent classification label is Class_0...
..       ...   ...                                                ...
195        2     2  \n\nThe text describes a process for producing...
196        0     0  \n\nThe patent classification label Class_01B ...
197        2     2  \n\nThe patent classification label is Class_0...
198        4     4  \n\nThe patent classification label Class_01F ...
199        2     2  \n\nClass_01D is the correct classification la...

[200 rows x 3 columns]

정답율: 84.50%


In [19]:
concat_df = pd.concat([eval_df,df], axis=1)
concat_df = concat_df.drop(['true'], axis=1)
concat_df.to_csv("temp11.csv")

In [15]:
for i in range(len(label_dict)):

    chat = [
    # {"role": "system", "content":"You are an AI that generates classification labels for patents. Read the text and classify them correctly. Let's Think Step by Step."},
    {"role": "user", "content": f'What is the patent classification label "{label_dict[i]}"?  Explain it.'}
    ]


    # Chat 형식에 맞춰 토큰화
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
    input_ids = inputs.to(device)

    # 텍스트 생성
    with torch.no_grad():
        generated_ids = model.lm_model.generate(
            input_ids,
            max_length=1024,
            num_return_sequences=1,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            # repetition_penalty = 1,
            top_p=0.95,
            # temperature=0.1
        )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(generated_text.split('assistant')[-1])
    print("___"*10)

/root/easymaker/custom-conda-envs/llm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/easymaker/custom-conda-envs/llm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




In the patent classification system, Class_01B is a subclass of Class 01, which is the main class for "Chemical and Chemical Products".

Class 01B is a more specific subclass, and it is defined as "Organic compounds".

In other words, Class_01B patents relate to inventions that involve the synthesis, properties, or uses of organic compounds, which are typically carbon-based molecules that contain hydrogen and often other elements such as oxygen, nitrogen, and sulfur.

Examples of inventions that might be classified under Class_01B include:

* New organic compounds with specific properties or uses
* Methods for synthesizing organic compounds
* Processes for purifying or modifying organic compounds
* Uses of organic compounds in various industries, such as pharmaceuticals, agrochemicals, or materials science.

Patent examiners and classifiers use the patent classification system to quickly identify the technical subject matter of a patent application and to group related inventions tog